In [3]:
import re
import nltk
from nltk import WordNetLemmatizer as wnl
nltk.download("wordnet")
nltk.download("omw-1.4")
import csv


[nltk_data] Downloading package wordnet to /home/nitin/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nitin/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [53]:
class flost():
    __metaclass__ = re,wnl
    def __init__(self,db):
        self.db = db
        self.wnl = wnl()

        # print(self.db.readlines())

    def index_words(self):
        index = {}
        self.db.execute('select MainTag,Lostid from LOST')
        print(self.db)
        for item in self.db:
            # print(index)
            desc = item[0]
            # print(line)
            words = re.findall(r'\w+',desc) # change to fetchall for sql db
            for word in words:
                word = word.lower()
                if word in index:
                    index[word].append(item[1])
                else:
                    index[word]=[item[1]]
        print(index)
        return index
    
    def remove_stop_words(self,index):
        stop_words = {'a', 'an', 'the', 'and', 'or', 'in', 'on', 'at'}
        for stop_word in stop_words:
            if stop_word in index:
                del index[stop_word]
        return index
    
    def stemmer(self,index):
        stm_idx={}
        for word,count in index.items():
            stm_word=self.wnl.lemmatize(word)
            if stm_word in stm_idx:
                stm_idx[stm_word]+=count
            else:
                stm_idx[stm_word]=count
        return stm_idx
    
    def stemQueries(self,query):
        stm_query = []
        for word in query:
            stm_word = self.wnl.lemmatize(word)
            stm_query.append(word)

        return stm_query
            
    
    def search(self,query,index):
        queryWords= re.findall(r'\w+',query.lower())
        stemmedQueries = self.stemQueries(queryWords)
        results = {}
        for word in stemmedQueries:
            if word in index:
                results[word] = index[word]
        return results

    def getItems(self,query):
        F_index = self.index_words()
        F_index = self.remove_stop_words(F_index)
        F_index = self.stemmer(F_index)
        results = self.search(query,F_index)
        return results
    



In [55]:
import mysql.connector

query = "laptop asus"

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="LAF"
)

mycursor = mydb.cursor()

fl = flost(mycursor)
matches = fl.getItems(query)
print(matches)

CMySQLCursor: select MainTag,Lostid from LOST
{'electronics': [1], 'laptop': [2, 3, 4], 'asus': [2], 'rog': [2], 'flow': [2], 'hp': [3, 4], 'omen': [3], 'spectre': [4]}
{'laptop': [2, 3, 4], 'asus': [2]}
